In [61]:
# This Fantasy Premier League project was/is inspired by an article published by Paul Corcoran's on Medium 
# "A data driven approach to become a better FPL manager" - https://blog.devgenius.io/using-data-to-become-a-better-fpl-manager-2c4d178d6107"
# The idea is to scrape data from the Fantasy Premier League API and generate a Streamlit tool to present the data in tabular form

In [62]:
# Import relevant packages
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler
import warnings
import json
warnings.filterwarnings('ignore')

In [63]:
# Connect to the Fantasy Premier League API - this endpoint responds with player specific data
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
res = requests.get(url)

In [64]:
# Check the status of the connection
if res.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {res.status_code}.')
# This code should return the text “The API request was successful” if the connection made was successful

The API request was successful.


In [65]:
# Store the output in json format (typically a dictionary)
json = res.json()

In [66]:
# Use the .keys() method to return the keys of the dictionary 
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [67]:
# We are interested in the following fields and need to transform each into a pd.DataFrame
# teams (contains the name and id of each team in the current season) 
teams_df = pd.DataFrame(json['teams'])
# elements_types_df (mapping for player positions)
elements_types_df = pd.DataFrame(json['element_types'])
# elements_df (all of the player stats) 
elements_df = pd.DataFrame(json['elements'])

In [68]:
# Anther way to do this
# Grab the data behind the FPL site
# data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
# Convert the data into a table
# dataTable = pd.DataFrame(data['elements'])
# List the variables
# dataTable.columns
# Select a few columns and show the top ten players by xG per 90
# dataTable[['web_name', 'points_per_game', 'total_points']].sort_values('points_per_game', ascending=False).head(10)

In [69]:
# Back to our project
# List the variables available in elements_df (the various player statistics)
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [70]:
# In order to avoid confusion we rename elements_df as players_df and select the variables that we are interested in

players_df = elements_df[
    ['web_name', 'first_name', 'second_name', 'team', 'element_type', 'now_cost', 'total_points', 'bonus', 'points_per_game', 'form',   
     'minutes', 'starts', 'starts_per_90', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'selected_by_percent', 
     'transfers_in', 'goals_scored', 'expected_goals', 'expected_goals_per_90', 'assists', 'expected_assists', 'expected_assists_per_90', 
     'expected_goal_involvements', 'expected_goal_involvements_per_90', 'goals_conceded', 'goals_conceded_per_90',
     'expected_goals_conceded', 'expected_goals_conceded_per_90', 'clean_sheets', 'clean_sheets_per_90', 'saves', 'saves_per_90', 
     'penalties_saved']]

# Some columns need to be converted from a string to a number (float)

cols_to_convert = ['now_cost', 'total_points', 'bonus', 'points_per_game', 'form', 'minutes', 'starts', 'starts_per_90', 
                   'selected_by_percent', 'transfers_in', 'goals_scored', 'expected_goals', 'expected_goals_per_90', 'assists', 
                   'expected_assists', 'expected_assists_per_90', 'expected_goal_involvements', 'expected_goal_involvements_per_90', 
                   'goals_conceded', 'goals_conceded_per_90', 'expected_goals_conceded', 'expected_goals_conceded_per_90', 
                   'clean_sheets', 'clean_sheets_per_90', 'saves', 'saves_per_90', 'penalties_saved']

players_df[cols_to_convert] = players_df[cols_to_convert].astype(float)

# Concat the first and second name variables into a new variable (Player)

players_df['Player'] = players_df['first_name'] + ' ' + players_df['second_name']


In [71]:
# List the variables in the players_df
# players_df.columns

In [72]:
# Print the players_df dataframe
players_df

,web_name,first_name,second_name,team,element_type,now_cost,total_points,bonus,points_per_game,form,...,goals_conceded,goals_conceded_per_90,expected_goals_conceded,expected_goals_conceded_per_90,clean_sheets,clean_sheets_per_90,saves,saves_per_90,penalties_saved,Player
0,Balogun,Folarin,Balogun,1,4,45.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,Folarin Balogun
1,Cédric,Cédric,Alves Soares,1,2,40.0,10.0,0.0,1.2,0.0,...,3.0,1.21,2.83,1.14,0.0,0.00,0.0,0.0,0.0,Cédric Alves Soares
2,M.Elneny,Mohamed,Elneny,1,3,45.0,6.0,0.0,1.2,0.0,...,2.0,1.62,1.29,1.05,0.0,0.00,0.0,0.0,0.0,Mohamed Elneny
3,Fábio Vieira,Fábio,Ferreira Vieira,1,3,55.0,40.0,2.0,1.8,0.0,...,5.0,0.90,5.28,0.95,2.0,0.36,0.0,0.0,0.0,Fábio Ferreira Vieira
4,Gabriel,Gabriel,dos Santos Magalhães,1,2,50.0,146.0,15.0,3.8,0.0,...,43.0,1.14,41.84,1.10,14.0,0.37,0.0,0.0,0.0,Gabriel dos Santos Magalhães
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,N.Semedo,Nélson,Cabral Semedo,20,2,45.0,75.0,5.0,2.1,0.0,...,43.0,1.47,50.02,1.71,8.0,0.27,0.0,0.0,0.0,Nélson Cabral Semedo
594,Toti,Toti António,Gomes,20,2,45.0,43.0,4.0,2.5,0.0,...,14.0,1.29,20.87,1.92,4.0,0.37,0.0,0.0,0.0,Toti António Gomes
595,Boubacar Traore,Boubacar,Traoré,20,3,45.0,14.0,0.0,1.4,0.0,...,9.0,2.00,7.62,1.69,0.0,0.00,0.0,0.0,0.0,Boubacar Traoré
596,Cunha,Matheus,Santos Carneiro Da Cunha,20,4,55.0,39.0,6.0,2.3,0.0,...,12.0,1.12,16.76,1.57,5.0,0.47,0.0,0.0,0.0,Matheus Santos Carneiro Da Cunha


In [73]:
# The format of the now_cost variable needs to be converted (in line with the Fantasy Premier League site values)
players_df['now_cost'] = (players_df['now_cost'] / 10).apply(lambda x: '{:.1f}'.format(x))

In [74]:
cols_to_convert = ['now_cost']
players_df[cols_to_convert] = players_df[cols_to_convert].astype(float)

In [75]:
# Check to data type for each variable
print(players_df.dtypes) 

web_name                              object
first_name                            object
second_name                           object
team                                   int64
element_type                           int64
now_cost                             float64
total_points                         float64
bonus                                float64
points_per_game                      float64
form                                 float64
minutes                              float64
starts                               float64
starts_per_90                        float64
chance_of_playing_next_round         float64
chance_of_playing_this_round          object
selected_by_percent                  float64
transfers_in                         float64
goals_scored                         float64
expected_goals                       float64
expected_goals_per_90                float64
assists                              float64
expected_assists                     float64
expected_a

In [76]:
# Add a calculated columns which will determine ROI per million spent on the player
players_df["ROI_Per_Million"] = (players_df["total_points"] / players_df["now_cost"])

In [77]:
# You can map the team and position names by using the map() method
players_df['team'] = players_df.team.map(teams_df.set_index('id').name)
players_df['position'] = players_df.element_type.map(elements_types_df.set_index('id').singular_name)
# Filter out players who have not played any minutes
players_df = players_df[players_df['minutes'] != 0]

In [78]:
# Aggregate the team totals
team_stats = players_df.groupby('team').agg({'total_points': 'sum', 'bonus': 'sum', 'goals_scored': 'sum','assists': 'sum','expected_goals': 'sum','expected_assists': 'sum'}).sort_values('total_points',ascending=False).reset_index()
# Add a couple of calculated columns
team_stats['performance_xG'] = team_stats['goals_scored'] - team_stats['expected_goals']
team_stats['performance_xA'] = team_stats['assists'] - team_stats['expected_assists']
columns = ['team_name','team_total_points','team_bonus','team_goals_scored','team_assists','team_expected_goals','team_expected_assists','team_performance_xG','team_performance_xA']
# rename these columns to avoid clashing with merge
team_stats.columns = columns
# Rearrange the columns to make more practical sense
team_stats = team_stats[['team_name','team_total_points','team_bonus','team_goals_scored','team_expected_goals','team_performance_xG','team_assists','team_expected_assists','team_performance_xA']]
# Rename columns
team_stats.rename(columns = {'team_name':'team', 'team_total_points':'points', 'team_bonus':'bonus', 'team_goals_scored':'goals', 'team_expected_goals':'xG', 'team_performance_xG':'goals - xG', 'team_assists':'assists', 'team_expected_assists':'xA', 'team_performance_xA':'assists - xA'}, inplace = True)

In [79]:
# Check your work
team_stats

,team,points,bonus,goals,xG,goals - xG,assists,xA,assists - xA
0,Arsenal,2407.0,214.0,104.0,93.99,10.01,82.0,53.93,28.07
1,Man City,1972.0,188.0,85.0,72.74,12.26,81.0,54.88,26.12
2,Newcastle,1921.0,170.0,65.0,74.81,-9.81,57.0,44.69,12.31
3,Liverpool,1788.0,148.0,67.0,76.17,-9.17,62.0,52.25,9.75
4,Man Utd,1770.0,165.0,58.0,66.06,-8.06,52.0,44.63,7.37
5,Brentford,1735.0,138.0,55.0,58.17,-3.17,50.0,30.42,19.58
6,Spurs,1728.0,136.0,79.0,63.99,15.01,65.0,38.64,26.36
7,Aston Villa,1578.0,131.0,45.0,47.91,-2.91,48.0,32.96,15.04
8,Brighton,1529.0,120.0,50.0,56.10,-6.10,56.0,38.59,17.41
9,Fulham,1400.0,109.0,46.0,43.41,2.59,40.0,21.13,18.87


In [80]:
# Subset the data based on player positions
defenders_df = players_df[players_df['position'] == 'Defender']
midfielders_df = players_df[players_df['position'] == 'Midfielder']
forwards_df = players_df[players_df['position'] == 'Forward']
goalkeepers_df = players_df[players_df['position'] == 'Goalkeeper']


In [81]:
# Subset each position dataframe with specific metrics
goalkeepers_df = goalkeepers_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_conceded','expected_goals_conceded','clean_sheets','saves','saves_per_90']]
goalkeepers_df['performance_xG_def'] = goalkeepers_df['goals_conceded'] - goalkeepers_df['expected_goals_conceded']
# Position newly-calculated metric within the dataframe
goalkeepers_df = goalkeepers_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_conceded','expected_goals_conceded','performance_xG_def','clean_sheets','saves','saves_per_90']]
defenders_df = defenders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','goals_conceded','expected_goals_conceded','clean_sheets']]
defenders_df['performance_xG_def'] = defenders_df['goals_conceded'] - defenders_df['expected_goals_conceded']
# Position newly-calculated metric within the dataframe
defenders_df = defenders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','goals_conceded','expected_goals_conceded','performance_xG_def','clean_sheets']]
midfielders_df = midfielders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','expected_goals_conceded']]
midfielders_df['performance_xG_off'] = midfielders_df['goals_scored'] - midfielders_df['expected_goals']
# Position newly-calculated metric within the dataframe
midfielders_df = midfielders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','performance_xG_off','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','expected_goals_conceded']]
forwards_df = forwards_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90']]
forwards_df['performance_xG_off'] = forwards_df['goals_scored'] - forwards_df['expected_goals']
# Position newly-calculated metric within the dataframe
forwards_df = forwards_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','performance_xG_off','expected_goals_per_90','assists','expected_assists','expected_assists_per_90']]


In [82]:
# Rename columns in order to make practical sense to the end user
goalkeepers_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_conceded':'conceded', 'expected_goals_conceded':'xGA', 'performance_xG_def':'conceded - xGA', 'clean_sheets':'clean sheets', 'saves':'saves', 'saves_per_90':'saves per 90'}, inplace = True)

In [83]:
# Rename columns in order to make practical sense to the end user
defenders_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90', 'expected_goal_involvements':'expected goal involvements', 'expected_goal_involvements_per_90':'expected goal involvement per 90', 'goals_conceded':'conceded', 'expected_goals_conceded':'xGA', 'performance_xG_def':'conceded - xGA', 'clean_sheets':'clean sheets'}, inplace = True)

In [84]:
# Rename columns in order to make practical sense to the end user
midfielders_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'performance_xG_off':'goals - xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90', 'expected_goals_conceded':'xGA'}, inplace = True)

In [85]:
# Rename columns in order to make practical sense to the end user
forwards_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'performance_xG_off':'goals - xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90'}, inplace = True)

In [86]:
# Export each dataframe to an individual CSV file
team_stats.to_csv('teams.csv',index=False)
goalkeepers_df.to_csv('goalkeepers.csv',index=False)
defenders_df.to_csv('defenders.csv',index=False)
midfielders_df.to_csv('midfielders.csv',index=False)
forwards_df.to_csv('forwards.csv',index=False)